In [1]:
import ast
import json
import re
from typing import List

import pandas as pd
import numpy as np
from src.module.processing import preprocessing as process

In [56]:
 def explode_json_column(dataframe: pd.DataFrame, column_name: str) -> pd.DataFrame:
    # Convertir les chaînes de caractères en dictionnaires Python
    dataframe[column_name] = dataframe[column_name].apply(ast.literal_eval)

    # Exploser le contenu de la colonne spécifiée
    exploded_df = pd.json_normalize(dataframe[column_name])

    # Renommer les colonnes si nécessaire (ici, on garde les noms originaux)
    exploded_df.columns = [f"{col}" for col in exploded_df.columns]

    return exploded_df

In [66]:
def replace_missing_with_mean(dataframe: pd.DataFrame) -> pd.DataFrame:
    for column in dataframe.select_dtypes(include=['float', 'int']).columns:
        mean_value = round(dataframe[column].mean(), 2)
        dataframe[column] = dataframe[column].fillna(mean_value)
    return dataframe

In [88]:
def replace_missing_with_mode(dataframe: pd.DataFrame) -> pd.DataFrame:
    for column in dataframe.select_dtypes(include=['object']).columns:
        # Calculer le mode de la colonne
        mode_value = dataframe[column].mode()[0]  # [0] pour récupérer la première valeur en cas d'égalité
        # Remplacer les valeurs manquantes par le mode
        dataframe[column] = dataframe[column].fillna(mode_value)
    return dataframe

In [7]:
path = "../data/cleaning/liste_fiches_technical_details_cleaning.csv"

In [21]:
df = pd.read_csv(path, sep=";")
df.head()

,Marque,Modele,Annee,Vehicule,Prix,Date Publication,Resumer,Dimensions,Weight,Habitability,Tires,Engine,Transmission,Performance,Consumption,Immatriculation
0,Ineos,Grenadier,2024,Ineos Grenadier 3.0 T 286ch Fieldmaster Edition,82490.0,2022-04-01,"{'energie': 'Essence', 'puissance_commerciale'...","{'longueur': 4.9, 'largeur': 1.93, 'hauteur': ...","{'poids_a_vide': 2669, 'ptac': 3500, 'ptra': 7...","{'nombre_de_places': 5, 'volume_de_coffre': 11...","{'types_de_pneumatiques': '4x4', 'materiau_des...","{'Nom_du_moteur': 3.0, 'Energie': 'Essence', '...",{'Boite_de_vitesses': 'Automatique 8 rapports'...,"{'Vitesse_maximale': 160, '0_a_100_km/h': 8.6,...","{'Mixte': 14.4, 'Emission_de_CO2': 325, 'Immat...",7e546927-d3e3-477f-8971-b0cd70187264
1,Mercedes-Benz,Classe C,2004,Mercedes-Benz Classe C II (W203) 240 V6 Elegance,36865.0,2004-04-01,"{'energie': 'Essence', 'puissance_commerciale'...","{'longueur': 4.52, 'largeur': 1.72, 'hauteur':...","{'poids_a_vide': 1535, 'ptac': 2015, 'ptra': 3...","{'nombre_de_places': 5, 'volume_de_coffre': 46...","{'types_de_pneumatiques': 'Classique', 'taille...","{'Nom_du_moteur': '240V6', 'Energie': 'Essence...","{'Boite_de_vitesses': 'Mécanique 6 rapports', ...","{'Vitesse_maximale': 235, '0_a_100_km/h': 9.2,...","{'Cycle_urbain': 16.0, 'Extra_urbain': 7.8, 'M...",fb2f74bc-f6e0-4624-967c-78d9466accfa
2,Jaguar,S-Type,2005,Jaguar S-Type 2.7D Bi-turbo,41700.0,2004-06-01,"{'energie': 'Diesel', 'puissance_commerciale':...","{'longueur': 4.91, 'largeur': 1.82, 'hauteur':...","{'poids_a_vide': 1722, 'ptac': 2255, 'ptra': 4...","{'nombre_de_places': 5, 'volume_de_coffre': 40...","{'types_de_pneumatiques': 'Classique', 'materi...","{'Nom_du_moteur': '2.7DV6207', 'Energie': 'Die...","{'Boite_de_vitesses': 'Mécanique 6 rapports', ...","{'Vitesse_maximale': 230, '0_a_100_km/h': 8.5,...","{'Cycle_urbain': 9.7, 'Extra_urbain': 5.7, 'Mi...",cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,Bmw,Série 5,2003,BMW Série 5 IV (E60) 530iA 231ch Premiere,44000.0,2003-07-01,"{'energie': 'Essence', 'puissance_commerciale'...","{'longueur': 4.84, 'largeur': 1.85, 'hauteur':...","{'poids_a_vide': 1580, 'ptac': 2065, 'ptra': 4...","{'nombre_de_places': 5, 'volume_de_coffre': 52...","{'types_de_pneumatiques': 'Classique', 'taille...","{'Nom_du_moteur': '3.0i', 'Energie': 'Essence'...",{'Boite_de_vitesses': 'Automatique 6 rapports'...,"{'Vitesse_maximale': 245, '0_a_100_km/h': 7.1,...","{'Cycle_urbain': 14.2, 'Extra_urbain': 7.5, 'M...",0285f39e-16ab-410d-939c-37de56206408
4,Opel,Combo,2006,Opel Combo Tour 1.7 CDTI100 Arizona,18600.0,2005-07-01,"{'energie': 'Diesel', 'puissance_commerciale':...","{'longueur': 4.32, 'largeur': 1.68, 'hauteur':...","{'poids_a_vide': 1290, 'ptac': 1855, 'ptra': 2...","{'nombre_de_places': 5, 'volume_de_coffre': 45...","{'types_de_pneumatiques': 'Classique', 'materi...","{'Nom_du_moteur': '1.7CDI100', 'Energie': 'Die...","{'Boite_de_vitesses': 'Mécanique 5 rapports', ...","{'Vitesse_maximale': 170, '0_a_100_km/h': 12.5...","{'Cycle_urbain': 6.6, 'Extra_urbain': 4.4, 'Mi...",3ea13155-3031-4ade-836f-c72c593b671c


In [24]:
df['Resumer'] = df['Resumer'].apply(ast.literal_eval)

In [46]:
resumer_df = pd.json_normalize(df['Resumer'])
resumer_df.columns = [f"{col}" for col in resumer_df.columns]

In [47]:
resumer_df

,energie,puissance_commerciale,puissance_fiscale,consommation_mixte,emission_de_co2,boite_de_vitesses,carrosserie,date_de_fin_de_commercialisation,Immatriculation,Object_Folder_Resumer
0,Essence,286.0,0,14.4,325.0,Automatique,4*4/SUV/Crossovers,-,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Resumer
1,Essence,NaN,12,10.8,NaN,Manuelle,Berlines,08/06/2005,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Resumer
2,Diesel,NaN,13,7.1,NaN,Manuelle,Berlines,01/06/2005,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Resumer
3,Essence,NaN,15,9.9,NaN,Automatique,Berlines,16/03/2005,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Resumer
4,Diesel,NaN,6,5.2,140.0,Manuelle,Break 5 portes,01/02/2012,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Resumer
...,...,...,...,...,...,...,...,...,...,...
6305,Diesel,NaN,4,4.1,109.0,Manuelle,Compactes,01/04/2010,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Resumer
6306,Diesel,NaN,5,NaN,100.0,Automatique,Compactes,03/07/2015,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Resumer
6307,Diesel,110.0,7,5.9,NaN,Manuelle,Berlines,01/06/2003,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Resumer
6308,Diesel,NaN,4,3.3,88.0,Manuelle,Compactes,23/01/2014,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Resumer


In [40]:
# resumer_df[
#     (resumer_df['puissance_fiscale'] == 0) &
#     (resumer_df['consommation_mixte'] <= 14.4)
#     ]


In [48]:
# Remplacer les valeurs manquantes par la moyenne arrondie à 2 chiffres pour chaque colonne
resumer_df['puissance_commerciale'] = resumer_df['puissance_commerciale'].fillna(round(resumer_df['puissance_commerciale'].mean(), 2))
resumer_df['puissance_fiscale'] = resumer_df['puissance_fiscale'].fillna(round(resumer_df['puissance_fiscale'].mean(), 2))
resumer_df['consommation_mixte'] = resumer_df['consommation_mixte'].fillna(round(resumer_df['consommation_mixte'].mean(), 2))
resumer_df['emission_de_co2'] = resumer_df['emission_de_co2'].fillna(round(resumer_df['emission_de_co2'].mean(), 2))

In [52]:
resumer_df.drop(columns=['Object_Folder_Resumer'], inplace=True)

In [53]:
resumer_df.to_csv("../data/cleaning/relation/resumer_data.csv", index=False, header=True)

In [54]:
resumer_df

,energie,puissance_commerciale,puissance_fiscale,consommation_mixte,emission_de_co2,boite_de_vitesses,carrosserie,date_de_fin_de_commercialisation,Immatriculation
0,Essence,286.0,0,14.40,325.0,Automatique,4*4/SUV/Crossovers,-,7e546927-d3e3-477f-8971-b0cd70187264
1,Essence,157.8,12,10.80,144.0,Manuelle,Berlines,08/06/2005,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,Diesel,157.8,13,7.10,144.0,Manuelle,Berlines,01/06/2005,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,Essence,157.8,15,9.90,144.0,Automatique,Berlines,16/03/2005,0285f39e-16ab-410d-939c-37de56206408
4,Diesel,157.8,6,5.20,140.0,Manuelle,Break 5 portes,01/02/2012,3ea13155-3031-4ade-836f-c72c593b671c
...,...,...,...,...,...,...,...,...,...
6305,Diesel,157.8,4,4.10,109.0,Manuelle,Compactes,01/04/2010,21b9fe07-34e8-448f-a360-f087ca3cd2d3
6306,Diesel,157.8,5,5.56,100.0,Automatique,Compactes,03/07/2015,dcc25760-5e24-4e17-a365-cb73e3c89108
6307,Diesel,110.0,7,5.90,144.0,Manuelle,Berlines,01/06/2003,489ee77e-d482-4e7a-b89b-f478926222b3
6308,Diesel,157.8,4,3.30,88.0,Manuelle,Compactes,23/01/2014,c9ad51ec-af8e-4072-a5d4-112b845a0d59


In [57]:
df.columns

Index(['Marque', 'Modele', 'Annee', 'Vehicule', 'Prix', 'Date Publication',
       'Resumer', 'Dimensions', 'Weight', 'Habitability', 'Tires', 'Engine',
       'Transmission', 'Performance', 'Consumption', 'Immatriculation'],
      dtype='object')

In [58]:
df_dimension = explode_json_column(dataframe=df, column_name='Dimensions')

In [59]:
df_dimension

,longueur,largeur,hauteur,empattement,reservoir,porte_a_faux_avant,porte_a_faux_arriere,voies_avant,voies_arriere,garde_au_sol,angle_dattaque,angle_ventral,angle_de_fuite,Immatriculation,Object_Folder_Dimensions,coefficient_daerodynamisme,hauteur_avec_barres_de_toit
0,4.90,1.93,2.05,2.92,90.0,0.887,0.874,1.645,1.645,264.0,35.5,28.2,36.1,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Dimensions,NaN,NaN
1,4.52,1.72,1.43,2.72,62.0,NaN,NaN,1.500,1.470,NaN,NaN,NaN,NaN,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Dimensions,"0,270 Cx",NaN
2,4.91,1.82,1.45,2.91,70.0,NaN,NaN,1.534,1.542,NaN,NaN,NaN,NaN,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Dimensions,NaN,NaN
3,4.84,1.85,1.47,2.89,70.0,NaN,NaN,1.558,1.582,NaN,NaN,NaN,NaN,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Dimensions,"0,280 Cx",NaN
4,4.32,1.68,1.80,2.72,52.0,NaN,NaN,1.417,1.440,NaN,NaN,NaN,NaN,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Dimensions,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6305,3.99,1.64,1.50,2.46,45.0,0.851,0.679,1.436,1.426,NaN,NaN,NaN,NaN,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Dimensions,NaN,NaN
6306,4.63,1.78,1.44,2.70,50.0,0.915,1.016,1.548,1.545,NaN,NaN,NaN,NaN,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Dimensions,NaN,NaN
6307,4.52,1.71,1.43,2.63,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Dimensions,NaN,NaN
6308,4.00,1.71,1.49,2.51,45.0,NaN,NaN,1.485,1.478,NaN,NaN,NaN,NaN,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Dimensions,NaN,NaN


In [60]:
df_dimension.drop(columns=['Object_Folder_Dimensions', 'coefficient_daerodynamisme', 'hauteur_avec_barres_de_toit'], inplace=True)

In [61]:
df_dimension.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6310 entries, 0 to 6309
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   longueur              5643 non-null   float64
 1   largeur               5559 non-null   float64
 2   hauteur               5477 non-null   float64
 3   empattement           5667 non-null   float64
 4   reservoir             5556 non-null   float64
 5   porte_a_faux_avant    3678 non-null   float64
 6   porte_a_faux_arriere  3742 non-null   float64
 7   voies_avant           4262 non-null   float64
 8   voies_arriere         4260 non-null   float64
 9   garde_au_sol          2579 non-null   float64
 10  angle_dattaque        696 non-null    float64
 11  angle_ventral         550 non-null    float64
 12  angle_de_fuite        702 non-null    float64
 13  Immatriculation       6310 non-null   object 
dtypes: float64(13), object(1)
memory usage: 690.3+ KB


In [67]:
df_dimension_clean = replace_missing_with_mean(df_dimension)

In [68]:
df_dimension_clean.head()

,longueur,largeur,hauteur,empattement,reservoir,porte_a_faux_avant,porte_a_faux_arriere,voies_avant,voies_arriere,garde_au_sol,angle_dattaque,angle_ventral,angle_de_fuite,Immatriculation
0,4.90,1.93,2.05,2.92,90.0,0.887,0.874,1.645,1.645,264.00,35.50,28.20,36.10,7e546927-d3e3-477f-8971-b0cd70187264
1,4.52,1.72,1.43,2.72,62.0,0.880,0.960,1.500,1.470,156.56,21.06,17.76,22.69,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,4.91,1.82,1.45,2.91,70.0,0.880,0.960,1.534,1.542,156.56,21.06,17.76,22.69,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,4.84,1.85,1.47,2.89,70.0,0.880,0.960,1.558,1.582,156.56,21.06,17.76,22.69,0285f39e-16ab-410d-939c-37de56206408
4,4.32,1.68,1.80,2.72,52.0,0.880,0.960,1.417,1.440,156.56,21.06,17.76,22.69,3ea13155-3031-4ade-836f-c72c593b671c


In [69]:
df_dimension_clean.to_csv("../data/cleaning/relation/dimension_data.csv", index=False, header=True)

In [72]:
df_weigth = explode_json_column(dataframe=df, column_name='Weight')

In [73]:
df_weigth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6310 entries, 0 to 6309
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   poids_a_vide             5142 non-null   float64
 1   ptac                     5213 non-null   float64
 2   ptra                     3659 non-null   float64
 3   charge_utile             3805 non-null   float64
 4   poids_tracte_freine      4546 non-null   float64
 5   poids_tracte_non_freine  3706 non-null   float64
 6   Immatriculation          6310 non-null   object 
 7   Object_Folder_Weight     6310 non-null   object 
dtypes: float64(6), object(2)
memory usage: 394.5+ KB


In [75]:
df_weigth.drop(columns=['Object_Folder_Weight'], inplace=True)

In [76]:
df_weigth_clean = replace_missing_with_mean(df_weigth)
df_weigth_clean.head()

,poids_a_vide,ptac,ptra,charge_utile,poids_tracte_freine,poids_tracte_non_freine,Immatriculation
0,2669.0,3500.0,7000.0,831.0,3500.0,750.00,7e546927-d3e3-477f-8971-b0cd70187264
1,1535.0,2015.0,3515.0,480.0,1500.0,750.00,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,1722.0,2255.0,4105.0,533.0,1850.0,750.00,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,1580.0,2065.0,4065.0,485.0,2000.0,750.00,0285f39e-16ab-410d-939c-37de56206408
4,1290.0,1855.0,2855.0,565.0,1000.0,702.73,3ea13155-3031-4ade-836f-c72c593b671c


In [77]:
df_weigth_clean.to_csv("../data/cleaning/relation/weigth_data.csv", index=False, header=True)

In [78]:
df_habitability = explode_json_column(dataframe=df, column_name='Habitability')

In [79]:
df_habitability

,nombre_de_places,volume_de_coffre,volume_de_coffre_utile,hauteur_de_seuil_de_chargement,longueur_utile,largeur_utile,Immatriculation,Object_Folder_Habitability
0,5.0,1152.0,2035.0,1039.0,1645.0,1064.0,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Habitability
1,5.0,465.0,1510.0,NaN,NaN,NaN,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Habitability
2,5.0,400.0,810.0,NaN,NaN,NaN,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Habitability
3,5.0,520.0,NaN,NaN,NaN,NaN,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Habitability
4,5.0,455.0,2700.0,573.0,NaN,1211.0,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Habitability
...,...,...,...,...,...,...,...,...
6305,5.0,300.0,1163.0,NaN,NaN,NaN,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Habitability
6306,5.0,470.0,NaN,NaN,NaN,NaN,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Habitability
6307,NaN,NaN,NaN,NaN,NaN,NaN,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Habitability
6308,5.0,285.0,1100.0,NaN,NaN,NaN,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Habitability


In [81]:
df_habitability.drop(columns=['Object_Folder_Habitability', 'hauteur_de_seuil_de_chargement', 'longueur_utile', 'largeur_utile'], inplace=True)

In [82]:
df_habitability

,nombre_de_places,volume_de_coffre,volume_de_coffre_utile,Immatriculation
0,5.0,1152.0,2035.0,7e546927-d3e3-477f-8971-b0cd70187264
1,5.0,465.0,1510.0,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,5.0,400.0,810.0,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,5.0,520.0,NaN,0285f39e-16ab-410d-939c-37de56206408
4,5.0,455.0,2700.0,3ea13155-3031-4ade-836f-c72c593b671c
...,...,...,...,...
6305,5.0,300.0,1163.0,21b9fe07-34e8-448f-a360-f087ca3cd2d3
6306,5.0,470.0,NaN,dcc25760-5e24-4e17-a365-cb73e3c89108
6307,NaN,NaN,NaN,489ee77e-d482-4e7a-b89b-f478926222b3
6308,5.0,285.0,1100.0,c9ad51ec-af8e-4072-a5d4-112b845a0d59


In [83]:
df_habitability_clean = replace_missing_with_mean(df_habitability)
df_habitability_clean.head()

,nombre_de_places,volume_de_coffre,volume_de_coffre_utile,Immatriculation
0,5.0,1152.0,2035.00,7e546927-d3e3-477f-8971-b0cd70187264
1,5.0,465.0,1510.00,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,5.0,400.0,810.00,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,5.0,520.0,1559.08,0285f39e-16ab-410d-939c-37de56206408
4,5.0,455.0,2700.00,3ea13155-3031-4ade-836f-c72c593b671c


In [84]:
df_habitability_clean.to_csv("../data/cleaning/relation/habitability_data.csv", index=False, header=True)

In [86]:
df_tires = explode_json_column(dataframe=df, column_name='Tires')

In [87]:
df_tires

,types_de_pneumatiques,materiau_des_jantes,taille_des_roues_avant,taille_des_roues_arriere,type_de_roues_de_secours,Immatriculation,Object_Folder_Tires
0,4x4,Aluminium,265/70 R17,265/70 R17,Normale,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Tires
1,Classique,NaN,205/55 R16,205/55 R16,NaN,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Tires
2,Classique,Aluminium,235/50 R17,235/50 R17,Galette,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Tires
3,Classique,NaN,225/55 R16,225/55 R16,NaN,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Tires
4,Classique,Aluminium,185/60 R15,185/60 R15,Normale,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Tires
...,...,...,...,...,...,...,...
6305,Classique,Aluminium,165/70 R14,165/70 R14,NaN,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Tires
6306,Eté,Aluminium,205/55 R16,205/55 R16,Kit anti-crevaison,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Tires
6307,NaN,NaN,NaN,NaN,NaN,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Tires
6308,Toutes saisons,Aluminium,185/65 R15,185/65 R15,NaN,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Tires


In [89]:
df_tires.drop(columns=['Object_Folder_Tires'], inplace=True)

In [90]:
df_tires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6310 entries, 0 to 6309
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   types_de_pneumatiques     5199 non-null   object
 1   materiau_des_jantes       4901 non-null   object
 2   taille_des_roues_avant    5453 non-null   object
 3   taille_des_roues_arriere  5452 non-null   object
 4   type_de_roues_de_secours  4381 non-null   object
 5   Immatriculation           6310 non-null   object
dtypes: object(6)
memory usage: 295.9+ KB


In [91]:
df_tires_clean = replace_missing_with_mode(df_tires)

In [92]:
df_tires_clean

,types_de_pneumatiques,materiau_des_jantes,taille_des_roues_avant,taille_des_roues_arriere,type_de_roues_de_secours,Immatriculation
0,4x4,Aluminium,265/70 R17,265/70 R17,Normale,7e546927-d3e3-477f-8971-b0cd70187264
1,Classique,Aluminium,205/55 R16,205/55 R16,Kit anti-crevaison,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,Classique,Aluminium,235/50 R17,235/50 R17,Galette,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,Classique,Aluminium,225/55 R16,225/55 R16,Kit anti-crevaison,0285f39e-16ab-410d-939c-37de56206408
4,Classique,Aluminium,185/60 R15,185/60 R15,Normale,3ea13155-3031-4ade-836f-c72c593b671c
...,...,...,...,...,...,...
6305,Classique,Aluminium,165/70 R14,165/70 R14,Kit anti-crevaison,21b9fe07-34e8-448f-a360-f087ca3cd2d3
6306,Eté,Aluminium,205/55 R16,205/55 R16,Kit anti-crevaison,dcc25760-5e24-4e17-a365-cb73e3c89108
6307,Eté,Aluminium,205/55 R16,205/55 R16,Kit anti-crevaison,489ee77e-d482-4e7a-b89b-f478926222b3
6308,Toutes saisons,Aluminium,185/65 R15,185/65 R15,Kit anti-crevaison,c9ad51ec-af8e-4072-a5d4-112b845a0d59


In [93]:
df_tires_clean.to_csv("../data/cleaning/relation/tires_data.csv", index=False, header=True)

In [85]:
df.columns

Index(['Marque', 'Modele', 'Annee', 'Vehicule', 'Prix', 'Date Publication',
       'Resumer', 'Dimensions', 'Weight', 'Habitability', 'Tires', 'Engine',
       'Transmission', 'Performance', 'Consumption', 'Immatriculation'],
      dtype='object')

In [94]:
df_engine = explode_json_column(dataframe=df, column_name='Engine')

In [95]:
df_engine

,Nom_du_moteur,Energie,Architecture,Alimentation,Injection,Cylindree,Au_regime_de,Couple_maxi,Nombre_de_soupapes,Rapport_volumetrique,Norme_anti-pollution,Disposition_du_moteur,Puissance_reelle_maxi_ch,Puissance_reelle_maxi_kW,Alesage,Course
0,3.0,Essence,Six cylindres en ligne,Turbo,Injection directe essence,2998.0,4750.0,450.0,24.0,11.0,Euro 6,Longitudinale Avant,286.0,210.0,82.0,94.6
1,240V6,Essence,Six cylindres en V,Atmosphérique,Injection multipoints,2597.0,4500.0,240.0,18.0,10.5,Euro 4,Longitudinale Avant,170.0,125.0,89.9,68.2
2,2.7DV6207,Diesel,Six cylindres en V,Bi-Turbos à géométrie variable,Injection directe à rampe commune,2720.0,1900.0,435.0,24.0,17.3,NaN,Longitudinale Avant,208.0,153.0,81.0,88.0
3,3.0i,Essence,Six cylindres en ligne,Atmosphérique,Injection multipoints,2979.0,3500.0,300.0,24.0,10.2,NaN,Longitudinale Avant,231.0,170.0,84.0,89.6
4,1.7CDI100,Diesel,Quatre cylindres en ligne,Turbo à géométrie variable,Injection directe à rampe commune,1686.0,2300.0,240.0,16.0,18.4,Euro 4,Transversale Avant,100.0,74.0,79.0,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6305,1.4DI80,Diesel,Trois cylindres en ligne,Turbo Basse Pression,Injection directe par injecteurs pompes,1422.0,2200.0,195.0,6.0,19.5,Euro 4,Transversale Avant,80.0,59.0,79.5,95.5
6306,180CDI,Diesel,Quatre cylindres en ligne,Turbo à géométrie variable,Injection directe à rampe commune,1461.0,4000.0,260.0,8.0,NaN,Euro 5,Transversale Avant,109.0,80.0,NaN,NaN
6307,110D-4D,Diesel,Quatre cylindres en ligne,Turbo,Injection,1995.0,NaN,NaN,16.0,NaN,NaN,NaN,110.0,81.0,NaN,NaN
6308,1.3CDI95,Diesel,Quatre cylindres en ligne,Turbo Haute Pression,Injection directe à rampe commune,1248.0,1750.0,190.0,16.0,18.0,Euro 5,Transversale Avant,95.0,70.0,69.6,82.0


In [96]:
df_engine_clean = replace_missing_with_mode(df_engine)
df_engine_clean = replace_missing_with_mean(df_engine)

In [97]:
df_engine_clean

,Nom_du_moteur,Energie,Architecture,Alimentation,Injection,Cylindree,Au_regime_de,Couple_maxi,Nombre_de_soupapes,Rapport_volumetrique,Norme_anti-pollution,Disposition_du_moteur,Puissance_reelle_maxi_ch,Puissance_reelle_maxi_kW,Alesage,Course
0,3.0,Essence,Six cylindres en ligne,Turbo,Injection directe essence,2998.0,4750.00,450.00,24.0,11.00,Euro 6,Longitudinale Avant,286.0,210.0,82.00,94.60
1,240V6,Essence,Six cylindres en V,Atmosphérique,Injection multipoints,2597.0,4500.00,240.00,18.0,10.50,Euro 4,Longitudinale Avant,170.0,125.0,89.90,68.20
2,2.7DV6207,Diesel,Six cylindres en V,Bi-Turbos à géométrie variable,Injection directe à rampe commune,2720.0,1900.00,435.00,24.0,17.30,Euro 6,Longitudinale Avant,208.0,153.0,81.00,88.00
3,3.0i,Essence,Six cylindres en ligne,Atmosphérique,Injection multipoints,2979.0,3500.00,300.00,24.0,10.20,Euro 6,Longitudinale Avant,231.0,170.0,84.00,89.60
4,1.7CDI100,Diesel,Quatre cylindres en ligne,Turbo à géométrie variable,Injection directe à rampe commune,1686.0,2300.00,240.00,16.0,18.40,Euro 4,Transversale Avant,100.0,74.0,79.00,86.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6305,1.4DI80,Diesel,Trois cylindres en ligne,Turbo Basse Pression,Injection directe par injecteurs pompes,1422.0,2200.00,195.00,6.0,19.50,Euro 4,Transversale Avant,80.0,59.0,79.50,95.50
6306,180CDI,Diesel,Quatre cylindres en ligne,Turbo à géométrie variable,Injection directe à rampe commune,1461.0,4000.00,260.00,8.0,13.88,Euro 5,Transversale Avant,109.0,80.0,81.78,87.35
6307,110D-4D,Diesel,Quatre cylindres en ligne,Turbo,Injection,1995.0,3497.79,294.79,16.0,13.88,Euro 6,Transversale Avant,110.0,81.0,81.78,87.35
6308,1.3CDI95,Diesel,Quatre cylindres en ligne,Turbo Haute Pression,Injection directe à rampe commune,1248.0,1750.00,190.00,16.0,18.00,Euro 5,Transversale Avant,95.0,70.0,69.60,82.00


In [98]:
df_engine_clean.to_csv("../data/cleaning/relation/engine_data.csv", index=False, header=True)

In [100]:
df_transmission = explode_json_column(dataframe=df, column_name='Transmission')
df_transmission

,Boite_de_vitesses,Mode_de_transmission
0,Automatique 8 rapports,Transmission intégrale permanente
1,Mécanique 6 rapports,Propulsion
2,Mécanique 6 rapports,Propulsion
3,Automatique 6 rapports,Propulsion
4,Mécanique 5 rapports,Traction
...,...,...
6305,Mécanique 5 rapports,Traction
6306,Mécanique robotisée 7 rapports à double embrayage,Traction
6307,Mécanique 5 rapports,Traction
6308,Mécanique 5 rapports,Traction


In [101]:
df_transmission_clean = replace_missing_with_mode(df_transmission)

In [102]:
df_transmission_clean.to_csv("../data/cleaning/relation/transmission_data.csv", index=False, header=True)

In [103]:
df_performance = explode_json_column(dataframe=df, column_name='Performance')
df_performance

,Vitesse_maximale,0_a_100_km/h,Immatriculation,Object_Folder_Performance,0_a_1000_m_DA
0,160.0,8.6,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Performance,NaN
1,235.0,9.2,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Performance,NaN
2,230.0,8.5,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Performance,NaN
3,245.0,7.1,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Performance,"27,3 s"
4,170.0,12.5,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Performance,NaN
...,...,...,...,...,...
6305,172.0,13.2,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Performance,NaN
6306,205.0,11.9,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Performance,NaN
6307,195.0,11.4,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Performance,NaN
6308,177.0,12.3,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Performance,NaN


In [104]:
df_performance.drop(columns=['Object_Folder_Performance', '0_a_1000_m_DA'], inplace=True)

In [107]:
df_performance_clean = replace_missing_with_mode(df_performance)
df_performance_clean = replace_missing_with_mean(df_performance)
df_performance_clean

,Vitesse_maximale,0_a_100_km/h,Immatriculation
0,160.0,8.6,7e546927-d3e3-477f-8971-b0cd70187264
1,235.0,9.2,fb2f74bc-f6e0-4624-967c-78d9466accfa
2,230.0,8.5,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9
3,245.0,7.1,0285f39e-16ab-410d-939c-37de56206408
4,170.0,12.5,3ea13155-3031-4ade-836f-c72c593b671c
...,...,...,...
6305,172.0,13.2,21b9fe07-34e8-448f-a360-f087ca3cd2d3
6306,205.0,11.9,dcc25760-5e24-4e17-a365-cb73e3c89108
6307,195.0,11.4,489ee77e-d482-4e7a-b89b-f478926222b3
6308,177.0,12.3,c9ad51ec-af8e-4072-a5d4-112b845a0d59


In [108]:
df_performance_clean.to_csv("../data/cleaning/relation/performance_data.csv", index=False, header=True)

In [109]:
df_consumption = explode_json_column(dataframe=df, column_name='Consumption')
df_consumption

,Mixte,Emission_de_CO2,Immatriculation,Object_Folder_Consumption,Cycle_urbain,Extra_urbain
0,14.4,325.0,7e546927-d3e3-477f-8971-b0cd70187264,Vehiculs/Models/INEOS/Consumption,NaN,NaN
1,10.8,259.0,fb2f74bc-f6e0-4624-967c-78d9466accfa,Vehiculs/Models/MERCEDES-BENZ/Consumption,16.0,7.8
2,7.1,189.0,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,Vehiculs/Models/JAGUAR/Consumption,9.7,5.7
3,9.9,245.0,0285f39e-16ab-410d-939c-37de56206408,Vehiculs/Models/BMW/Consumption,14.2,7.5
4,5.2,140.0,3ea13155-3031-4ade-836f-c72c593b671c,Vehiculs/Models/OPEL/Consumption,6.6,4.4
...,...,...,...,...,...,...
6305,4.1,109.0,21b9fe07-34e8-448f-a360-f087ca3cd2d3,Vehiculs/Models/SKODA/Consumption,5.3,3.4
6306,NaN,100.0,dcc25760-5e24-4e17-a365-cb73e3c89108,Vehiculs/Models/MERCEDES-BENZ/Consumption,NaN,NaN
6307,5.9,158.0,489ee77e-d482-4e7a-b89b-f478926222b3,Vehiculs/Models/TOYOTA/Consumption,8.0,4.8
6308,3.3,88.0,c9ad51ec-af8e-4072-a5d4-112b845a0d59,Vehiculs/Models/OPEL/Consumption,3.8,3.1


In [110]:
df_consumption.drop(columns=['Object_Folder_Consumption'], inplace=True)

In [111]:
df_consumption_clean = replace_missing_with_mode(df_consumption)
df_consumption_clean = replace_missing_with_mean(df_consumption)
df_consumption_clean

,Mixte,Emission_de_CO2,Immatriculation,Cycle_urbain,Extra_urbain
0,14.40,325.0,7e546927-d3e3-477f-8971-b0cd70187264,8.13,5.3
1,10.80,259.0,fb2f74bc-f6e0-4624-967c-78d9466accfa,16.00,7.8
2,7.10,189.0,cab0aab1-ba72-4778-b7f4-d1efaa37d5a9,9.70,5.7
3,9.90,245.0,0285f39e-16ab-410d-939c-37de56206408,14.20,7.5
4,5.20,140.0,3ea13155-3031-4ade-836f-c72c593b671c,6.60,4.4
...,...,...,...,...,...
6305,4.10,109.0,21b9fe07-34e8-448f-a360-f087ca3cd2d3,5.30,3.4
6306,6.28,100.0,dcc25760-5e24-4e17-a365-cb73e3c89108,8.13,5.3
6307,5.90,158.0,489ee77e-d482-4e7a-b89b-f478926222b3,8.00,4.8
6308,3.30,88.0,c9ad51ec-af8e-4072-a5d4-112b845a0d59,3.80,3.1


In [112]:
df_consumption_clean.to_csv("../data/cleaning/relation/consumption_data.csv", index=False, header=True)